# Chapter 05 - Model Deployment

## Overview

The goal of this notebook is to show how you can use Azure Machine Learning service to deploy the machine learning models that have been trained using Automated Machine Learning (AutoML).
If you have already trained the model, you can skip earlier steps, and focus on Step 3. Step 3 shows how you can deploy the model using Azure Container Instances (ACI), and then use the deployed endpoint for prediction.

The notebook is organized as follows:

1. Acquire and Prepare Data
2. Automated ML
3. Deploy Model



## 1. Acquire and Prepare Data
For this notebook, we will use the NASA Prognostics Center's Turbo-Fan Failure dataset.  It is located here: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan

We have it as .txt file in the same folder. We read it into a Pandas DataFrame.
Note the headers were not in the space seperated txt file, so we assign them from the ReadMe in the zip file. In pandas we use read_csv with the delimiter option, even with a space delimited file.

In [20]:
import pandas as pd
train = pd.read_csv("train_FD001.txt", delimiter="\s|\s\s", index_col=False, engine='python', names=['unit','cycle','os1','os2','os3','sm1','sm2','sm3','sm4','sm5','sm6','sm7','sm8','sm9','sm10','sm11','sm12','sm13','sm14','sm15','sm16','sm17','sm18','sm19','sm20','sm21'])

Take a quick look at the data

In [21]:
train.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm12,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21
0,1,1,-0.00,-0.00,100.00,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.42,0.03,392,2388,100.00,39.06,23.42
1,1,2,0.00,-0.00,100.00,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.43,0.03,392,2388,100.00,39.00,23.42
2,1,3,-0.00,0.00,100.00,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.42,0.03,390,2388,100.00,38.95,23.34
3,1,4,0.00,0.00,100.00,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.37,0.03,392,2388,100.00,38.88,23.37
4,1,5,-0.00,-0.00,100.00,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.43,0.03,393,2388,100.00,38.90,23.40


Our dataset has a number of units in it, with each engine flight listed as a cycle. The cycles count up until the engine fails. What we would like to predict is the no. of cycles until failure. 
So we need to calculate a new column called RUL, or Remaining Useful Life.  It will be the last cycle value minus each cycle value per unit.

In [22]:
# Assign ground truth
def assignrul(df):
    maxi = df['cycle'].max()
    df['rul'] = maxi - df['cycle']
    return df
    

train_new = train.groupby('unit').apply(assignrul)

train_new.columns

Index(['unit', 'cycle', 'os1', 'os2', 'os3', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5',
       'sm6', 'sm7', 'sm8', 'sm9', 'sm10', 'sm11', 'sm12', 'sm13', 'sm14',
       'sm15', 'sm16', 'sm17', 'sm18', 'sm19', 'sm20', 'sm21', 'rul'],
      dtype='object')

Now our dataframe has the 'RUL' column.  Predicting this value will be the objective of this exercise.

In [23]:
train_new.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21,rul
0,1,1,-0.00,-0.00,100.00,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.42,0.03,392,2388,100.00,39.06,23.42,191
1,1,2,0.00,-0.00,100.00,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.43,0.03,392,2388,100.00,39.00,23.42,190
2,1,3,-0.00,0.00,100.00,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.42,0.03,390,2388,100.00,38.95,23.34,189
3,1,4,0.00,0.00,100.00,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.37,0.03,392,2388,100.00,38.88,23.37,188
4,1,5,-0.00,-0.00,100.00,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.43,0.03,393,2388,100.00,38.90,23.40,187


First note that the sensor measurements do seem to be changing as we near 0 RUL. This implies that we should be able to make a model that will be useful enough for business value.

We are now ready to train a model on this data using Automated ML.

## 1. Preparing the Azure Machine Learning Workspace

In [ ]:
!pip install --upgrade azureml-sdk

Import Azure ML libs for automated ML

In [13]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.core.model import Model

Provide your Machine Learning Workspace credentials to run AutoML. You will need to perform Microsoft's MFA. Please follow the manual auth instructions.

In [2]:
subscription_id = "<Your SubscriptionId>" #you should be owner or contributor
resource_group = "automl_nasa_newrg" #you should be owner or contributor
workspace_name = "automatedml_nasa_aznb" #your workspace name
workspace_region = "West US 2" #your region

Create Azure ML workspace

In [ ]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

Write config to cluster

In [17]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Define the experiment name

In [ ]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-predictive-rul'
project_folder = './sample_projects/automl-demo-predmain'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

Create training data

In [24]:
# put training data into X and Y arrays
X_train = train_new.iloc[:,2:26].values
y_train = train_new.iloc[:,26:27].values.astype(int).flatten()

Format data into arrays that AutoML will use to train models.  Below is one row of X and Y as an example.

In [25]:
X_train[0]

array([-7.00000e-04, -4.00000e-04,  1.00000e+02,  5.18670e+02,
        6.41820e+02,  1.58970e+03,  1.40060e+03,  1.46200e+01,
        2.16100e+01,  5.54360e+02,  2.38806e+03,  9.04619e+03,
        1.30000e+00,  4.74700e+01,  5.21660e+02,  2.38802e+03,
        8.13862e+03,  8.41950e+00,  3.00000e-02,  3.92000e+02,
        2.38800e+03,  1.00000e+02,  3.90600e+01,  2.34190e+01])

In [26]:
y_train[0]

191

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.3,
                                                    random_state=100)

X_valid, X_test, y_valid, y_test = train_test_split(X_valid,
                                                    y_valid,
                                                    test_size=0.3,
                                                    random_state=100)

X_train = pd.DataFrame(X_train)
X_valid = pd.DataFrame(X_valid)
X_test = pd.DataFrame(X_test)

In [28]:
X_test[0:1].values.tolist()

[[-0.0014,
  0.0004,
  100.0,
  518.67,
  642.93,
  1592.39,
  1407.61,
  14.62,
  21.61,
  553.5,
  2388.12,
  9059.72,
  1.3,
  47.65,
  521.36,
  2388.08,
  8133.52,
  8.4613,
  0.03,
  393.0,
  2388.0,
  100.0,
  38.6,
  23.3778]]

Enable Telemetry

In [29]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on. 


Now we are ready to configure automated ML.  We provide necessary information on: what we want to predict, what accuracy metric we want to use, how many models we want to try, and many other parameters.  AutoML will also automatically scale the data for us.

## 2. Getting started with Automated ML

You can use these params.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [31]:
##Local compute 
Automl_config = AutoMLConfig(task = 'regression',
                             primary_metric = 'r2_score',
                             iteration_timeout_minutes = 15,
                             iterations = 10,
                             max_cores_per_iteration = 1,
                             preprocess = False,
                             experiment_exit_score = 0.985,
                             X = X_train,
                             y = y_train,
                             X_valid = X_valid,
                             y_valid = y_valid,
                             #n_cross_validations = 3,
                             debug_log = 'automl_errors.log',
                             verbosity=logging.ERROR,
                             #model_explainability=True,
                             path=project_folder)

Finally we are ready to launch AutoML.  This step can take many minutes, but AutoML will give you updates as models are trained and evaluated by the metric we specified above.  AutoML also let us know which scaling method was used.  The information from each ML model training will be stored in the Experiment section of the ML Workspace, where we can review it through Azure Portal.

In [32]:
# Train multiple models using AutoML
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(Automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_d781db81-9575-404a-af20-b5f1d08198dc
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper LightGBM                 0:00:22       0.6113    0.6113
         1   MaxAbsScaler LightGBM                          0:00:17       0.6091    0.6113
         2   StandardScalerWrapper ExtremeRandomTrees       0:00:13       0.5432    0.6113
         3   RobustScaler LightG

### Visualize the run using the widget 

In [33]:
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

We now get the best model

In [34]:
# find the run with the highest accuracy value.
best_run, fitted_model = local_run.get_output()
print(best_run)

Run(Experiment: automl-predictive-rul,
Id: AutoML_d781db81-9575-404a-af20-b5f1d08198dc_9,
Type: None,
Status: Completed)


## 3. Deploy Model

In [35]:
# register best model in workspace
description = 'AutoML-RUL-Regression-20190507'
tags = None
model = local_run.register_model(description = description, tags = tags)

print(local_run.model_id) # This will be written to the script file later in the notebook.

Registering model AutoMLd781db819best
AutoMLd781db819best


After we register the model in our ML Workspace, it should be visible in Azure Portal.

Now we want to deploy the model as a REST API that we can feed a row or rows of "X" data to, and return the predicted 'RUL' value.  To accomplish this, we will build a container image in our AML Workspace and deploy that image as a Container instance in Azure's ACI service.  We will then obtain an IP address where we can submit data and receive back the predicted 'RUL' value.

There are 3 things we need: 
1. A score.py file that contains the init() and run() functions with instructions on how to load and socre with the model
2. A myenv.yml file that contains information on the python environment in which the model needs to run
3. Configurations for our images and our services, using functions provided by AzureML service.

The cells below help you set these up.   You will need to use the registered model name provided by the cell above.

In [36]:
%%writefile score.py
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path(model_name = '<<modelid>>') # this name is model.id of model that we want to deploy
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})

Overwriting score.py


In [37]:
experiment = Experiment(ws, experiment_name)
ml_run = AutoMLRun(experiment = experiment, run_id = local_run.id)
dependencies = ml_run.get_run_sdk_dependencies(iteration = 0)
for p in ['azureml-train-automl', 'azureml-sdk', 'azureml-core']:
    print('{}\t{}'.format(p, dependencies[p]))

No issues found in the SDK package versions.
azureml-train-automl	1.0.33
azureml-sdk	1.0.33
azureml-core	1.0.33.1


In [38]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','lightgbm'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

'myenv.yml'

In [40]:
# Substitute the actual version number in the environment file.
# This is not strictly needed in this notebook because the model should have been generated using the current SDK version.
# However, we include this in case this code is used on an experiment from a previous SDK version.

with open(conda_env_file_name, 'r') as cefr:
    content = cefr.read()

with open(conda_env_file_name, 'w') as cefw:
    cefw.write(content.replace(azureml.core.VERSION, dependencies['azureml-sdk']))

# Substitute the actual model id in the script file.

script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()

with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', local_run.model_id))

*If the above file does not show "- azureml-train-automl" in the pip section, the you need to add it manually*

Now configure the webservice.

Finally, configure the container image and deploy the service. Make sure the filenames match, your Workspace is in variable ws, and your model name is correct. It will create your containter image and deploy it as a webservice.

This process can take up to 10 minutes, so please be patient. You can check the progress bar periodically ...

In [41]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script = script_file_name,
                                 conda_file = conda_env_file_name,
                                 tags = {'area': "pred maint", 'type': "automl_regression"},
                                 description = "Image for automl predictive maintenance NASA")

image = Image.create(name = "automlpredmaintimage",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

if image.creation_state == 'Failed':
    print("Image build log at: " + image.image_build_log_uri)

Creating image
Running..............................................................
SucceededImage creation operation finished for image automlpredmaintimage:1, operation "Succeeded"


In [42]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "RUL",  "method" : "sklearn"}, 
                                               description='Predict RUL with Azure AutoML')

In [43]:
from azureml.core.webservice import Webservice

aci_service_name = 'automl-pred-maint'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-pred-maint
Creating service
Running...........................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


Just as a check, we can retrieve the URI for the scoring function.

In [44]:
print(aci_service.scoring_uri)

http://13.83.246.98:80/score


Let's check to see if the service is working.  Here we submit a single row of data from X_train to see if it returns a reasonable prediction.

In [45]:
import requests
import json

# send a random row from the test set to score
#random_index = np.random.randint(0, len(X_train)-1)
input_data = "{\"data\": " + str(X_test[1:2].values.tolist()) + "}" #str(list(X_train[0].reshape(1,-1)[0])) + "}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(aci_service.scoring_uri, input_data, headers=headers)

print("POST to url", aci_service.scoring_uri)
print("input data:", input_data)
print("label:", y_test[1:2])
print("prediction:", resp.text)

POST to url http://13.83.246.98:80/score
input data: {"data": [[-0.0006, -0.0004, 100.0, 518.67, 642.25, 1589.25, 1412.42, 14.62, 21.61, 553.88, 2388.15, 9043.21, 1.3, 47.45, 521.88, 2388.18, 8131.61, 8.4807, 0.03, 392.0, 2388.0, 100.0, 38.6, 23.2946]]}
label: [154]
prediction: "{\"result\": [118.17594443178578]}"


Here we see one engine evolving through many flights, or cycles.  As we approach failure, the rul declines to zero, as does the prediction.  This is a good example of how the predictive model can assist in estimate the future failure of the engine.

Note that the model does not perform well at high rul.  This is an acceptable outcome as the engine is far from failure.

To avoid any run-away Azure costs, we always delete un-necessary services when we are done.

In [ ]:
aci_service.delete()